# Reinforcement Learning ( Using MonteCarlo algorithem to play Black Jack)

## reseting and cleaning the variables and outputs

In [1]:
%reset -f
!rm *

'rm' is not recognized as an internal or external command,
operable program or batch file.


## Importing the Important pyton packages

In [1]:
import gym
import pandas as pd
import random
from collections import defaultdict
import numpy as np

## Creating and testing the Black Jack environment

In [8]:
'''Create a blackjack environment'''
env = gym.make('Blackjack-v1', render_mode='rgb_array')

In [ ]:
print(list(env.reset()))
'''False=  player does not have a usable ace, True= player has usable ace'''


[(8, 10, False), {}]


'False=  player does not have a usable ace, True= player has usable ace'

In [5]:
print(env.action_space)
print(env.observation_space)

Discrete(2)
Tuple(Discrete(32), Discrete(11), Discrete(2))


## Importing the policy to run the game

In [10]:
'''every visit monte carlo'''
'''0 is stand and 1 is hit'''


def policy(state):
    return 0 if state[0] > 19 else 1



num_timesteps = 100

def generate_episode(policy):
    episode = []
    state = list(env.reset()[0])
    for t in range(num_timesteps):
        action = policy(state)
        [next_state, reward, done, info, _]  = env.step(action)
        episode.append([state, action, reward])
        if done:
            episode.append([next_state])
            break
        state = list(next_state)
        
    
    return episode

generated_episode = generate_episode(policy)
print(generated_episode)
print(*generated_episode)

[[[15, 10, False], 1, 0.0], [[17, 10, False], 1, 0.0], [[20, 10, False], 0, 0.0], [(20, 10, False)]]
[[15, 10, False], 1, 0.0] [[17, 10, False], 1, 0.0] [[20, 10, False], 0, 0.0] [(20, 10, False)]


In [11]:
'''working with * to point to the state and rewared and action'''
poped_generated_episode = generated_episode[:-1]
print(poped_generated_episode)
states, actions, rewards = zip(*poped_generated_episode)
print(states)
print(rewards)
print(actions)

[[[15, 10, False], 1, 0.0], [[17, 10, False], 1, 0.0], [[20, 10, False], 0, 0.0]]
([15, 10, False], [17, 10, False], [20, 10, False])
(0.0, 0.0, 0.0)
(1, 1, 0)


In [12]:
'''working with numpy array to point to the state and reward and action'''
a = np.array(poped_generated_episode, dtype=list).transpose()
print(a)
states, actions, rewards = zip(a)
print(states)
print(rewards)
print(actions)

[[list([15, 10, False]) list([17, 10, False]) list([20, 10, False])]
 [1 1 0]
 [0.0 0.0 0.0]]
(array([list([15, 10, False]), list([17, 10, False]),
       list([20, 10, False])], dtype=object),)
(array([0.0, 0.0, 0.0], dtype=object),)
(array([1, 1, 0], dtype=object),)


In [19]:
'''every-visit monte carlo prediction'''
'''if I use 'human' render_mode it consumes a long time to run. Thus, I used 'rgb_array'.'''
def policy(state):
    return 0 if state[0] > 19 else 1

num_timesteps = 100

def generate_episode(policy):
    episode = []
    state = env.reset()[0]
    for t in range(num_timesteps):
        action = policy(state)
        [next_state, reward, done, info, _]  = env.step(action)
        episode.append([state, action, reward])
        if done:
            episode.append([next_state])
            break
        state = next_state
        
        # print(next_value)
    return episode

total_return = defaultdict(float)
N = defaultdict(int)
num_iterations = 50000
for i in range(num_iterations):
    episode = generate_episode(policy)[:-1]
    states, actions, rewards = zip(*episode)
    for t, state in enumerate(states):
        R = (sum(rewards[t:]))
        total_return[state] = total_return[state] + R
        N[state] = N[state] + 1
total_return = pd.DataFrame(total_return.items(),columns=['state', 'total_return'])
N = pd.DataFrame(N.items(),columns=['state', 'N'])
df = pd.merge(total_return, N, on="state")
df['value'] = df['total_return']/df['N']
df.head(50)

,state,total_return,N,value
0,"(12, 10, True)",-34.0,116,-0.293103
1,"(15, 10, True)",-72.0,206,-0.349515
2,"(15, 10, False)",-1302.0,1979,-0.657908
3,"(18, 10, False)",-1572.0,2124,-0.740113
4,"(17, 1, False)",-407.0,560,-0.726786
5,"(20, 1, False)",128.0,778,0.164524
6,"(12, 2, False)",-234.0,451,-0.518847
7,"(18, 2, False)",-376.0,505,-0.744554
8,"(10, 10, False)",-190.0,793,-0.239596
9,"(20, 10, False)",1310.0,2950,0.444068


In [20]:
'''epsilon-greedy monte carlo on_policy control'''
'''Initialize the dictionary for storing the Q values'''

Q = defaultdict(float)
total_return = defaultdict(float)
N = defaultdict(int)

'''Define the epsilon-greedy policy'''

def epsilon_greedy_policy(state,Q):
    epsilon = 0.9
    if random.uniform(0,1) < epsilon:
        return env.action_space.sample()
    else:
        return max(list(range(env.action_space.n)), key = lambda x: Q[(state,x)])
            
'''Generating an episode'''
num_timesteps = 100
def generate_episode(Q):
    episode = []
    state = env.reset()[0]
    for t in range(num_timesteps):
        action = epsilon_greedy_policy(state,Q)
        [next_state, reward, done, info, _]  = env.step(action)
        episode.append([state, action, reward])
        if done:
            break
        state = next_state
    return episode



'''Computing the optimal policy'''
num_iterations = 50000
for i in range(num_iterations):
    episode = generate_episode(Q)
    all_state_action_pairs = [(s, a) for (s,a,r) in episode]
    rewards = [r for (s,a,r) in episode]
    for t, (state, action,_) in enumerate(episode):
        if not (state, action) in all_state_action_pairs[0:t]:
            R = sum(rewards[t:])
            total_return[(state,action)] = total_return[(state,action)] + R
            N[(state, action)] += 1
            Q[(state,action)] = total_return[(state, action)] / N[(state, action)]
df = pd.DataFrame(Q.items(),columns=['state_action pair','value'])
df.head(60)

,state_action pair,value
0,"((18, 7, True), 0)",0.342857
1,"((13, 2, True), 1)",-0.076923
2,"((13, 2, False), 0)",-0.238095
3,"((20, 10, False), 1)",-0.944568
4,"((12, 10, False), 0)",-0.632391
5,"((14, 7, False), 1)",-0.534091
6,"((16, 7, False), 0)",-0.542857
7,"((14, 3, False), 0)",-0.223881
8,"((18, 10, False), 0)",-0.245658
9,"((10, 10, False), 0)",-0.580952


In [7]:
env.close()